In [7]:
import numpy as np
import pandas as pd
import torch
from pathlib import Path
import open3d as o3d
import os
from torch_geometric.data import InMemoryDataset, Data
from sklearn.model_selection import train_test_split, KFold
import random

In [11]:
class IMDataset(InMemoryDataset):
    def __init__(self, raw_data_root, dataset_root, basic_features_path, ids_root, target_name, gender, transform=None, pre_transform=None):
        """
        Creates the InMemoryDataset object, convenient structure for graph training.
        :param raw_data_root: path for the input mesh data 
        :param dataset_root: path where the in memory dataset will be written
        :param basic_features_path: path of the "basic_features.csv"
        :param ids_root: path of the file saving the ids of selected samples
        :param target_name: name of the feature to be predicted
            ["sex","bmi","age","weight","height"]
        :param gender: 0 for female, 1 for male
        """
        self.ids = np.load(ids_root)
        self.raw_data_root = raw_data_root
        self.basic_features_path = basic_features_path
        self.target_name = target_name

        if not os.path.exists(dataset_root):
            os.makedirs(dataset_root)
        super(IMDataset, self).__init__(root = dataset_root, transform = transform, pre_transform = pre_transform)
        if gender == 0: #female
            self.data, self.slices = torch.load(self.processed_paths[0])
        elif gender == 1: #male
            self.data, self.slices = torch.load(self.processed_paths[1])
 
    @property
    def raw_file_names(self):
        file_names = (pd.Series(self.ids).astype(str) + ".ply").values.tolist()
        return [os.path.join(str(Path(self.raw_data_root)), file_name) for file_name in file_names]

    @property
    def processed_file_names(self):
        processed_root = self.root + "/" + self.target_name

        if not os.path.exists(processed_root):
            os.makedirs(processed_root)

        return [processed_root + "/female_dataset.pt", processed_root + "/male_dataset.pt"]
 
    def download(self):
        pass
    
    def process(self):
        """
        Combines the vertex coordinates, edge connectivity indexes and chosen target feature.
        """
        male_data_list = []
        female_data_list = []

        if self.target_name == "vat":
            col = "22407-2.0"
        elif self.target_name == "asat":
            col = "22408-2.0"

        features = pd.read_csv(self.basic_features_path, usecols =["eid", "31-0.0", col])

        for file_path in self.raw_file_names:
            _id = int(os.path.splitext(os.path.basename(file_path))[0])
            _sex = int(features[features['eid']==_id]["31-0.0"].values[0]) #female: 0, male: 1
            _y = torch.tensor(features[features['eid']==_id][col].values).double()
            if torch.isnan(_y[0]):
                continue
            
            _mesh = o3d.io.read_triangle_mesh(file_path)
            _vertices = torch.from_numpy(np.asarray(_mesh.vertices)).double()
            _triangles = _mesh.triangles 

            """
            necessary to get the edge connectivity since the original data has only face connectivity
            we need [0,1],[1,2],[0,2],[1,0],[2,1],[2,0] instead of [0,1,2]
            including reverse as well to make the graph bi-directional
            """
            edge_list = []
            for t in _triangles:
                edge_list = edge_list + [[t[0], t[1]], [t[1], t[0]], [t[0], t[2]], [t[2], t[0]], [t[1], t[2]], [t[2], t[1]]]

            _edges = torch.from_numpy(np.unique(np.array(edge_list), axis=0)).long().permute(1, 0)

            if _sex == 0:
                female_data_list.append(Data(x=_vertices, edge_index=_edges, y=_y))
            else:
                male_data_list.append(Data(x=_vertices, edge_index=_edges, y=_y))
        
        data_female, slices_female = self.collate(female_data_list)
        data_male, slices_male = self.collate(male_data_list)
        #saves sex based datasets
        torch.save((data_female, slices_female), self.processed_paths[0])
        torch.save((data_male, slices_male), self.processed_paths[1])

In [12]:
REGISTERED_ROOT = "/vol/space/projects/ukbb/projects/silhouette/registered_5" # the path of the dir saving the .ply registered data
INMEMORY_ROOT = '/vol/space/projects/ukbb/projects/silhouette/imdataset/registered5_imdataset' # the root dir path to save all the artifacts ralated of the InMemoryDataset
FEATURES_PATH = "/vol/space/projects/ukbb/projects/silhouette/ukb668815_imaging.csv"   
IDS_PATH = "/vol/space/projects/ukbb/projects/silhouette/eids_filtered.npy"
TARGET = "all"

In [13]:
dataset_female = IMDataset(REGISTERED_ROOT, INMEMORY_ROOT, FEATURES_PATH, IDS_PATH, TARGET, 0)
dataset_male = IMDataset(REGISTERED_ROOT, INMEMORY_ROOT, FEATURES_PATH, IDS_PATH, TARGET, 1)

In [14]:
female_list = list(dataset_female)
male_list = list(dataset_male)

dev_female, test_female = train_test_split(female_list, test_size=1/6, random_state=42)
dev_male, test_male = train_test_split(male_list, test_size=1/6, random_state=42)

In [15]:
len(dev_female)

10906

In [16]:
len(dev_male)

10175

In [22]:
k = 4
kf = KFold(n_splits=k, shuffle=True)

for (train_index_female, val_index_female), (train_index_male, val_index_male) in zip(kf.split(dev_female), kf.split(dev_male)):
    print(len(train_index_female) + len(val_index_female))
    print(len(train_index_male) + len(val_index_male))


    # 划分数据集
    # train_female = [dev_female[i] for i in train_index_female]
    # val_female = [dev_female[i] for i in val_index_female]
    # train_male = [dev_male[i] for i in train_index_male]
    # val_male = [dev_male[i] for i in val_index_male]

    # print(len(train_female))

    

10906
10175
10906
10175
10906
10175
10906
10175


In [19]:
a = [0,1,2,3]
a[0:-1]

[0, 1, 2]

In [1]:
from codecarbon import EmissionsTracker

In [2]:
tracker = EmissionsTracker()

[codecarbon INFO @ 21:58:48] [setup] RAM Tracking...
[codecarbon INFO @ 21:58:48] [setup] GPU Tracking...
[codecarbon INFO @ 21:58:48] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:58:48] [setup] CPU Tracking...
[codecarbon WARNING @ 21:58:48] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:58:49] CPU Model on constant consumption mode: AMD Ryzen Threadripper 3960X 24-Core Processor
[codecarbon INFO @ 21:58:49] >>> Tracker's metadata:
[codecarbon INFO @ 21:58:49]   Platform system: Linux-5.4.0-148-generic-x86_64-with-glibc2.17
[codecarbon INFO @ 21:58:49]   Python version: 3.8.15
[codecarbon INFO @ 21:58:49]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 21:58:49]   Available RAM : 251.625 GB
[codecarbon INFO @ 21:58:49]   CPU count: 48
[codecarbon INFO @ 21:58:49]   CPU model: AMD Ryzen Threadripper 3960X 24-Core Processor
[codecarbon INFO @ 21:58:49]   GPU count: 1
[codecarbon INFO @ 21:58:49]   GPU model: 1 x Quadro RTX 8000
[codecarbon WA

<bound method Emissions.get_country_emissions of <codecarbon.core.emissions.Emissions object at 0x7f3b88e8eeb0>>